# Extract Data from GoFactory Gocator Data

Inputs:
- path to source directory including image/ and surface/ subdirectories
- image/ contains .jpg files for intensity images (uint8)
- surface contains .tar files that include Gocator profile data (uint16) and metadata for creating height maps and point clouds

Outputs: 
- new directory called surface_extract
- extracted profile raw data (uint16)
- metadata including reconstruction parameters
- hmap (3 channel uint8)
- blend: intensity image + hmap (3 channel uint8)
- warp: blend stretched to true aspect ratio
- pointcloud: pcd file from profile + metadata and intensity images

*Set input data path.*

In [ ]:
input_data_path='./'

*Add lmi_utils to your current path*

In [ ]:
import os
import sys
home_directory = os.path.expanduser('~')
sys.path.append(os.path.join(home_directory,'projects','LMI_AI_Solutions','lmi_utils'))

*Import dependencies*

In [ ]:
import glob
import tarfile
import cv2
from inspection_utils.profile_to_hmap import preprocess_hmap
from image_utils.img_stretch import stretch
import numpy as np
import json
import open3d

*Create a function to convert profile data and image data to a pointcloud that includes x,y,z, and rgb data for each point*

In [ ]:
def generate_pcd(profile,rgb_image,path):
    MISSING_PIXELS=0
    with open(os.path.join(path, "metadata.json"), "r") as f:
        metadata = json.load(f)
    resolution = metadata["resolution"]
    offset = metadata["offset"]
    shape = profile.shape
    x, y, z, rgb = [], [], [], []        
    i = 0
    for yi in range(shape[0]):
        for xi in range(shape[1]):
            if profile[yi][xi] != MISSING_PIXELS:
                x.append(offset[0] + xi * resolution[0])
                y.append(offset[1] + yi * resolution[1])
                z.append(offset[2] + profile[yi][xi] * resolution[2])
                rgb.append(rgb_image[yi][xi]/255.0)
                i += 1
    np_points = np.empty((i, 3))
    np_points[:, 0] = np.array(x)
    np_points[:, 1] = np.array(y)
    np_points[:, 2] = np.array(z)
    np_rgb=np.array(rgb) 
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(np_points)
    pcd.colors = open3d.utility.Vector3dVector(np_rgb)
    # fname=os.path.split(path)[1]
    fname='pointcloud'
    open3d.io.write_point_cloud(os.path.join(path, fname+'.pcd'), pcd)
    return resolution


*Set up directories*

In [ ]:
# Path Definitions
# Existing Directories
image_path=os.path.join(input_data_path,'image')
surface_path=os.path.join(input_data_path,'surface')
# New Directories
surface_extract_path=os.path.join(input_data_path,'surface_extract')
# Create New Directories
if not os.path.exists(surface_extract_path):
    os.makedirs(surface_extract_path,exist_ok=True)

*Generate files from input data*

In [ ]:
intensity_image_files=glob.glob(os.path.join(image_path,'*.jpg'))
for img_file in intensity_image_files:
    # Extract Gocator scans from tar file
    fname=os.path.split(img_file)[1].replace('gadget2d.jpg','gadget3d.tar')
    file_path=os.path.join(surface_path,fname)
    with tarfile.open(file_path, 'r:') as tar:
        dirname=fname.replace('.gadget3d.tar','')
        profile_dir=os.path.join(surface_extract_path,dirname)
        tar.extractall(path=profile_dir)
    print(f'[INFO].tar Extraction complete: {fname}')
    # Generate hmap from surface data
    profile_path=glob.glob(os.path.join(profile_dir,'*.png'))[0]
    img_p=cv2.imread(profile_path,-1)
    ind_nonzero=np.where(img_p!=0)
    ind_zero=np.where(img_p==0)
    max_height=img_p[ind_nonzero].max()
    hmap=preprocess_hmap(img_p,map_choice='rainbow-med',global_max=None,remove_outliers=True)
    img_bgr=cv2.cvtColor(hmap,cv2.COLOR_RGB2BGR)
    path=os.path.split(profile_path)
    fname='hmap.png'
    cv2.imwrite(os.path.join(path[0],fname),img_bgr)
    # Blend hmap with intensity image
    img_inten=cv2.imread(img_file)
    img_inten=cv2.cvtColor(img_inten,cv2.COLOR_BGR2RGB)
    alpha=0.9
    img_blend=cv2.addWeighted(img_inten,alpha,hmap,1-alpha, 0)
    fname='blend.png'
    cv2.imwrite(os.path.join(path[0],fname),cv2.cvtColor(img_blend,cv2.COLOR_RGB2BGR))
    # Generate pcd
    resolution=generate_pcd(img_p,img_inten,path[0])
    # Warp the image to true aspect ratio
    wh_stretch=resolution[1]/resolution[0]
    img_stretch=stretch(img_blend,wh_stretch)
    fname='warp.png'
    cv2.imwrite(os.path.join(path[0],fname),cv2.cvtColor(img_stretch,cv2.COLOR_RGB2BGR))